# Linear Regression Example

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 199 kB 43.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=51777da86d04dfa0308371be8ed544c6ee7e8febc6555e71161a4b6526b616db
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
training_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')
training_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [ ]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol = 'features', labelCol = 'label', predictionCol = 'prediction')

#fit the model
lrModel = lr.fit(training_data)

In [ ]:
#print the coefficients and intercept for linear regression
print("Coefficients : {}".format(str(lrModel.coefficients))) #for each feature
print('\n')
print("intercept : {}".format(str(lrModel.intercept)))

Coefficients : [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290002,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.695615180432293]


intercept : 0.14228558260358093


In [ ]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
trainingSummary.residuals.show()
print("RSME : {}".format(trainingSummary.rootMeanSquaredError))
print("r2 : {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176535|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
| 2.1228071931912336|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RSME : 10.16309157133015
r2 : 0.027839179518600154


## Train and Test Split

In [ ]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [ ]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
+----------

In [ ]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                347|
|   mean|-0.0722386866037906|
| stddev| 10.426214091441924|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [ ]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -16.08565904102149|(10,[0,1,2,3,4,5,...|
| -15.86200932757056|(10,[0,1,2,3,4,5,...|
|-15.780685032623301|(10,[0,1,2,3,4,5,...|
|-15.375857723312297|(10,[0,1,2,3,4,5,...|
|-14.822152909751189|(10,[0,1,2,3,4,5,...|
|-14.762758252931127|(10,[0,1,2,3,4,5,...|
|-14.328978509075442|(10,[0,1,2,3,4,5,...|
| -12.46765638103286|(10,[0,1,2,3,4,5,...|
|-11.904986902675114|(10,[0,1,2,3,4,5,...|
|-11.640549677888826|(10,[0,1,2,3,4,5,...|
|-11.039347808253828|(10,[0,1,2,3,4,5,...|
| -10.29566593602992|(10,[0,1,2,3,4,5,...|
|-10.233439586953153|(10,[0,1,2,3,4,5,...|
+----------

In [ ]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                154|
|   mean| 0.9984943128000394|
| stddev| 10.063466778212646|
|    min|-23.487440120936512|
|    max| 24.290551295953957|
+-------+-------------------+



In [ ]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [ ]:
correct_model = lr.fit(train_data)
test_result = correct_model.evaluate(test_data)
test_result.residuals.show()
print("RSME : {}".format(test_result.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|-22.290051288744614|
|-26.708345916213755|
| -20.22113977597685|
| -20.82028453400029|
|-19.700363065355877|
|  -19.8399974114709|
|-20.127679534609392|
|-15.333812135634194|
|-15.939396010930357|
|-18.465008838444625|
|-15.937523452546873|
|-14.533828451148027|
| -14.93238882437086|
|-15.971171472505276|
|-10.333298215799196|
|-11.312711974085154|
| -7.813879147592332|
| -7.899886867194808|
| -9.775576148384417|
| -8.310108441671403|
+-------------------+
only showing top 20 rows

RSME : 9.965613185154208


In [ ]:
predictions = correct_model.transform(unlabeled_data)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.1973888321918988|
|(10,[0,1,2,3,4,5,...|    3.75851998001768|
|(10,[0,1,2,3,4,5,...|  -2.616320640942492|
|(10,[0,1,2,3,4,5,...|  0.6082072750416168|
|(10,[0,1,2,3,4,5,...|-0.18419770891754952|
|(10,[0,1,2,3,4,5,...| 0.05723462185636324|
|(10,[0,1,2,3,4,5,...|  1.2817570617108078|
|(10,[0,1,2,3,4,5,...| -0.7518469053872967|
|(10,[0,1,2,3,4,5,...| 0.07738668335979856|
|(10,[0,1,2,3,4,5,...|   2.684323805821324|
|(10,[0,1,2,3,4,5,...|  0.5616657292345756|
|(10,[0,1,2,3,4,5,...| -0.2883244586031613|
|(10,[0,1,2,3,4,5,...|  0.1696305714397329|
|(10,[0,1,2,3,4,5,...|  1.6421929634298336|
|(10,[0,1,2,3,4,5,...|  -2.134358165233663|
|(10,[0,1,2,3,4,5,...| -0.5922749285899608|
|(10,[0,1,2,3,4,5,...|  -3.826670530296494|
|(10,[0,1,2,3,4,5,...| -3.1394609410590197|
|(10,[0,1,2,3,4,5,...| -0.5200897876455023|
|(10,[0,1,2,3,4,5,...|   -1.9233

# Linear Regression on Dataset

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
spark = SparkSession.builder.appName('lrExample').getOrCreate()

In [ ]:
data = spark.read.csv('Ecommerce_Customers.csv',
                      inferSchema = True, header = True)

In [ ]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [ ]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [ ]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [ ]:
for item in data.head():
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting up dataframe for Machine Learning

In [ ]:
#import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [ ]:
assembler = VectorAssembler(inputCols = ["Avg Session Length", "Time on App", "Time on Website","Length of Membership"],
                            outputCol = "features")
output = assembler.transform(data)
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [ ]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [ ]:
final_data = output.select("features", "Yearly Amount Spent")
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [ ]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                339|
|   mean|  498.2837274129786|
| stddev|  80.58905575657741|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [ ]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                161|
|   mean|  501.4834505359973|
| stddev|  76.76406427540446|
|    min|  275.9184206503857|
|    max|  700.9170916173961|
+-------+-------------------+



In [ ]:
#create linear regression model
lr = LinearRegression(labelCol = "Yearly Amount Spent")

#fit the model to the data 
lrModel = lr.fit(train_data)

# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.665399527255683,38.339595585315706,0.9830574686757073,61.661283476692255] Intercept: -1065.273432755636


In [ ]:
test_results = lrModel.evaluate(test_data)

test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  9.406579425531845|
| -11.56376284345913|
| -4.902171641183315|
|  11.02657913254859|
| -3.596645114349201|
|-13.128620238248175|
|  -20.9447864598759|
|  22.57727898806411|
| -9.265280010353422|
| -6.931574919965897|
| -5.910980345299606|
|-10.024552213509594|
|  3.072676705283641|
|-14.157107445847714|
|  18.04116481397682|
|  -4.26655067311691|
|-11.284900560855249|
|-2.0239443088382245|
| 0.5925358093898012|
| -2.752230831377119|
+-------------------+
only showing top 20 rows



In [ ]:
unlabeled_data = test_data.select('features')

predictions = lrModel.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|399.23377164709564|
|[30.3931845423455...|331.49263264665274|
|[30.4925366965402...|287.37341736109784|
|[30.7377203726281...| 450.7541630636813|
|[30.8794843441274...| 493.8032450992039|
|[31.0662181616375...| 462.0619134459225|
|[31.1239743499119...| 507.8918402996417|
|[31.2834474760581...| 569.2038104376034|
|[31.4474464941278...|427.86802210557744|
|[31.5147378578019...| 496.7440629164273|
|[31.5171218025062...| 281.8294009956853|
|[31.5261978982398...| 419.1190784058474|
|[31.5316044825729...| 433.4429290240789|
|[31.5741380228732...| 558.5663796064346|
|[31.6005122003032...| 461.1316866771201|
|[31.6253601348306...| 380.6034514300411|
|[31.8093003166791...| 548.0567999236964|
|[31.8186165667690...| 448.4426176789739|
|[31.8293464559211...| 384.5598021785852|
|[31.8530748017465...| 462.0373542937291|
+--------------------+------------

In [ ]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.203337192589297
MSE: 104.10808986567602
